In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json
import requests

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [4]:
from citipy import citipy

In [5]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

622

In [6]:
from config import weather_api_key
weather_api_key

'317febeb3bb45c35ce9927992a7c4f88'

In [7]:
url="http://api.openweathermap.org/data/2.5/weather?"
units="imperial"
#units=metrics for celsius 
city="London"

query_url=url+"appid="+weather_api_key+"&q="+city+"&units="+units
query_url

'http://api.openweathermap.org/data/2.5/weather?appid=317febeb3bb45c35ce9927992a7c4f88&q=London&units=imperial'

In [8]:
weather_response=requests.get(query_url)

weather_response.status_code

200

In [9]:
weather_json=weather_response.json()
weather_json

{'coord': {'lon': -0.1257, 'lat': 51.5085},
 'weather': [{'id': 802,
   'main': 'Clouds',
   'description': 'scattered clouds',
   'icon': '03n'}],
 'base': 'stations',
 'main': {'temp': 43.34,
  'feels_like': 33.31,
  'temp_min': 42.01,
  'temp_max': 45,
  'pressure': 1018,
  'humidity': 76},
 'visibility': 10000,
 'wind': {'speed': 12.66, 'deg': 260},
 'clouds': {'all': 40},
 'dt': 1610845933,
 'sys': {'type': 1,
  'id': 1414,
  'country': 'GB',
  'sunrise': 1610870248,
  'sunset': 1610900602},
 'timezone': 0,
 'id': 2643743,
 'name': 'London',
 'cod': 200}

In [10]:
weather_json['main']['temp']

43.34

In [11]:
weather_json['main']

{'temp': 43.34,
 'feels_like': 33.31,
 'temp_min': 42.01,
 'temp_max': 45,
 'pressure': 1018,
 'humidity': 76}

In [12]:
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=317febeb3bb45c35ce9927992a7c4f88


In [13]:
# Create an endpoint URL for a city.
city_url = url
city_weather = requests.get(city_url)
if city_weather.json():
    print(f"City Weather found.")
else:
    print(f"City weather not found.")

City Weather found.


In [14]:
city_weather = requests.get(city_url)
weather_json=weather_response.json()
weather_json

{'coord': {'lon': -0.1257, 'lat': 51.5085},
 'weather': [{'id': 802,
   'main': 'Clouds',
   'description': 'scattered clouds',
   'icon': '03n'}],
 'base': 'stations',
 'main': {'temp': 43.34,
  'feels_like': 33.31,
  'temp_min': 42.01,
  'temp_max': 45,
  'pressure': 1018,
  'humidity': 76},
 'visibility': 10000,
 'wind': {'speed': 12.66, 'deg': 260},
 'clouds': {'all': 40},
 'dt': 1610845933,
 'sys': {'type': 1,
  'id': 1414,
  'country': 'GB',
  'sunrise': 1610870248,
  'sunset': 1610900602},
 'timezone': 0,
 'id': 2643743,
 'name': 'London',
 'cod': 200}

In [15]:
boston_data=weather_json
lat = boston_data["coord"]["lat"]
lng = boston_data["coord"]["lon"]
max_temp = boston_data["main"]["temp_max"]
humidity = boston_data["main"]["humidity"]
clouds = boston_data["clouds"]["all"]
wind = boston_data["wind"]["speed"]
print(lat, lng, max_temp, humidity, clouds, wind)

51.5085 -0.1257 45 76 40 12.66


In [16]:
# Import the datetime module from the datetime library.
from datetime import datetime
# Get the date from the JSON file.
date = boston_data["dt"]
# Convert the UTC date to a date format with year, month, day, hours, minutes, and seconds.
datetime.utcfromtimestamp(date)

datetime.datetime(2021, 1, 17, 1, 12, 13)

In [17]:
datetime.utcfromtimestamp(date).strftime('%Y-%m-%d %H:%M:%S')

'2021-01-17 01:12:13'

In [18]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

Beginning Data Retrieval     
-----------------------------


In [19]:
# Loop through all the cities in the list.
city_data=[]
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

print()
# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Processing Record 1 of Set 1 | broken hill
Processing Record 2 of Set 1 | kirakira
Processing Record 3 of Set 1 | cherskiy
Processing Record 4 of Set 1 | samarai
Processing Record 5 of Set 1 | katsuura
Processing Record 6 of Set 1 | vardo
Processing Record 7 of Set 1 | tiksi
Processing Record 8 of Set 1 | aktash
Processing Record 9 of Set 1 | butaritari
Processing Record 10 of Set 1 | hobart
Processing Record 11 of Set 1 | cacapava do sul
Processing Record 12 of Set 1 | zhangye
Processing Record 13 of Set 1 | ushuaia
Processing Record 14 of Set 1 | tasiilaq
Processing Record 15 of Set 1 | saint-philippe
Processing Record 16 of Set 1 | yellowknife
Processing Record 17 of Set 1 | victoria
Processing Record 18 of Set 1 | chuy
Processing Record 19 of Set 1 | touba
Processing Record 20 of Set 1 | basco
Processing Record 21 of Set 1 | dikson
Processing Record 22 of Set 1 | pochutla
Processing Record 23 of Set 1 | kapaa
Processing Record 24 of Set 1 | belushya guba
City not found. Skipping...

Processing Record 38 of Set 4 | honningsvag
Processing Record 39 of Set 4 | nikolskoye
Processing Record 40 of Set 4 | huarmey
Processing Record 41 of Set 4 | kamloops
Processing Record 42 of Set 4 | hasaki
Processing Record 43 of Set 4 | rtishchevo
Processing Record 44 of Set 4 | taoudenni
Processing Record 45 of Set 4 | burns lake
Processing Record 46 of Set 4 | bhag
Processing Record 47 of Set 4 | karamay
City not found. Skipping...
Processing Record 48 of Set 4 | the valley
Processing Record 49 of Set 4 | miramar
Processing Record 50 of Set 4 | bose
Processing Record 1 of Set 5 | kralendijk
Processing Record 2 of Set 5 | narayangarh
Processing Record 3 of Set 5 | kumluca
Processing Record 4 of Set 5 | dali
Processing Record 5 of Set 5 | xingyi
Processing Record 6 of Set 5 | chengde
Processing Record 7 of Set 5 | salalah
Processing Record 8 of Set 5 | kudahuvadhoo
Processing Record 9 of Set 5 | zabol
Processing Record 10 of Set 5 | mataura
Processing Record 11 of Set 5 | lebu
Proces

Processing Record 32 of Set 8 | saskylakh
Processing Record 33 of Set 8 | beringovskiy
Processing Record 34 of Set 8 | tongren
Processing Record 35 of Set 8 | mogadishu
Processing Record 36 of Set 8 | svobodnyy
Processing Record 37 of Set 8 | capoterra
Processing Record 38 of Set 8 | mount isa
Processing Record 39 of Set 8 | jardim
Processing Record 40 of Set 8 | omsukchan
Processing Record 41 of Set 8 | los llanos de aridane
Processing Record 42 of Set 8 | longyearbyen
Processing Record 43 of Set 8 | udgir
Processing Record 44 of Set 8 | sanming
Processing Record 45 of Set 8 | moree
Processing Record 46 of Set 8 | scarborough
Processing Record 47 of Set 8 | old orchard beach
Processing Record 48 of Set 8 | coari
Processing Record 49 of Set 8 | oxelosund
Processing Record 50 of Set 8 | san jeronimo
Processing Record 1 of Set 9 | roma
Processing Record 2 of Set 9 | barra do garcas
Processing Record 3 of Set 9 | quatre cocos
Processing Record 4 of Set 9 | viet tri
Processing Record 5 of 

Processing Record 22 of Set 12 | athabasca
Processing Record 23 of Set 12 | muros
Processing Record 24 of Set 12 | tome-acu
City not found. Skipping...
Processing Record 25 of Set 12 | mao
Processing Record 26 of Set 12 | pringsewu
Processing Record 27 of Set 12 | fukue
Processing Record 28 of Set 12 | ketchikan
Processing Record 29 of Set 12 | tongchuan
Processing Record 30 of Set 12 | snyder
Processing Record 31 of Set 12 | raga
Processing Record 32 of Set 12 | acajutla
Processing Record 33 of Set 12 | ludvika
Processing Record 34 of Set 12 | bridlington
Processing Record 35 of Set 12 | swan hill
Processing Record 36 of Set 12 | sorland
Processing Record 37 of Set 12 | maragogi
Processing Record 38 of Set 12 | namatanai
Processing Record 39 of Set 12 | egvekinot
Processing Record 40 of Set 12 | freeport
Processing Record 41 of Set 12 | filotion
Processing Record 42 of Set 12 | klaksvik
Processing Record 43 of Set 12 | itoman
Processing Record 44 of Set 12 | san rafael del sur
Process

In [22]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Broken Hill,-31.9500,141.4333,68.00,45,93,16.11,AU,2021-01-17 01:19:12
1,Kirakira,-10.4544,161.9205,86.16,66,100,7.23,SB,2021-01-17 01:20:27
2,Cherskiy,68.7500,161.3000,-21.89,87,100,3.22,RU,2021-01-17 01:18:49
3,Samarai,-10.6167,150.6667,85.62,73,100,18.79,PG,2021-01-17 01:20:27
4,Katsuura,35.1333,140.3000,46.00,52,100,4.00,JP,2021-01-17 01:18:51
...,...,...,...,...,...,...,...,...,...
567,Xifeng,42.7372,124.7222,1.90,88,0,12.88,CN,2021-01-17 01:20:57
568,Zhanaozen,43.3000,52.8000,35.26,86,100,10.00,KZ,2021-01-17 01:19:08
569,Port Lincoln,-34.7333,135.8667,66.02,64,84,11.65,AU,2021-01-17 01:20:57
570,Patacamaya,-17.2333,-67.9167,46.31,79,100,7.43,BO,2021-01-17 01:20:57


In [25]:
new_column_order = ["City", "Country", "Date", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed"]
city_data_df=city_data_df[new_column_order]
city_data_df.head()

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Broken Hill,AU,2021-01-17 01:19:12,-31.9500,141.4333,68.00,45,93,16.11
1,Kirakira,SB,2021-01-17 01:20:27,-10.4544,161.9205,86.16,66,100,7.23
2,Cherskiy,RU,2021-01-17 01:18:49,68.7500,161.3000,-21.89,87,100,3.22
3,Samarai,PG,2021-01-17 01:20:27,-10.6167,150.6667,85.62,73,100,18.79
4,Katsuura,JP,2021-01-17 01:18:51,35.1333,140.3000,46.00,52,100,4.00


In [26]:
# Create the output file (CSV).
output_data_file = "weather_data/cities.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")